In [20]:
from Bio import SeqIO
import pandas as pd

In [2]:
for record in SeqIO.parse("/home/lihaicheng/data1/References/human/mit/chrM.fa", "fasta"):
    pass

In [46]:
ls = []
idx = 0
for i in range(40, len(record.seq)+ 35, 40):
    ls.append(str(record.seq[idx:i+20]))
    idx = i

In [48]:
with open("split.fa", "w") as f:
    for i,j in enumerate(ls):
        f.write(f">{i}\n")
        f.write(f"{j}\n")

```shell
 blastn -query split.fa -out split.blast -db ../project24/hg38 -outfmt 6   -num_threads 30 -evalue 0.05  -task blastn-short
```

variant

In [ ]:
chrm_gnomad = pd.read_csv("gnomad_mitochondrial_variants_M-1-16569_filtered.csv", comment="#")

In [ ]:
def work(x):
    return pd.Series(x.split("-"))

In [ ]:
id = chrm_gnomad["Variant ID"].apply(work)

chrm_gnomad = pd.concat([id,chrm_gnomad], axis=1)

In [ ]:
blast = pd.read_csv("split.blast", header=None, sep='\t')
tmp = pd.DataFrame(blast[0].value_counts()).reset_index()
tmp = tmp.sort_values("index")
tmp["idx"] = tmp["index"] * 40 
idx = list(tmp["idx"])
blast_count = list(tmp[0])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 示例数据：SNP的位置及其对应的突变频率（频率越低表示越保守）
snp_positions = [int(i) for i in chrm_gnomad[1]]
snp_frequencies = [float(i) for i in chrm_gnomad["Homoplasmic Allele Frequency"]]  # 这些频率可以替换成实际数据

# SNP位置排序
sorted_indices = np.argsort(snp_positions)
snp_positions = np.array(snp_positions)[sorted_indices]
snp_frequencies = np.array(snp_frequencies)[sorted_indices]

# 设置窗口大小，这里选择1000为窗口的长度，你可以根据需要调整
window_size = 40

# 计算加权SNP密度
def calculate_weighted_snp_density(snp_positions, snp_frequencies, window_size):
    densities = []
    max_position = max(snp_positions)
    
    # 对每一个窗口位置，计算窗口内SNP频率的加权密度
    for start in range(0, max_position, window_size):
        end = start + window_size
        # 获取当前窗口内的SNP位置和对应的频率
        snps_in_window = [i for i in range(len(snp_positions)) if start <= snp_positions[i] < end]
        
        # 如果窗口内有SNP，则计算该窗口的加权密度
        if snps_in_window:
            snp_count = len(snps_in_window)
            # 计算窗口内所有SNP频率的和（频率较高的SNP贡献较大）
            weighted_density = np.sum(snp_frequencies[snps_in_window])*snp_count
        else:
            weighted_density = 0  # 如果窗口内没有SNP，则密度为0
        
        densities.append(weighted_density)
    
    return densities, np.arange(0, max_position, window_size)


# 计算每个窗口的加权SNP密度
densities, window_positions = calculate_weighted_snp_density(snp_positions, snp_frequencies, window_size)

# 绘制加权SNP密度图
plt.figure(figsize=(80, 6)) 
plt.plot(window_positions, densities, label='frquency + count', color='b') 
plt.plot(idx, blast_count, label='blast + count', color='r') 
plt.axhline(y=1, color='k', linestyle='--', label='y=1') 
plt.xlabel('Genomic Position')
plt.ylabel('Weighted SNP Density (Lower is more conserved)')
plt.xticks(range(0, 17001, 100), rotation=45) 

plt.title('SNP Density Across Genomic Regions (Weighted by Frequency)')
plt.legend()
plt.savefig('mit.pdf', dpi=300)
plt.show()